#### Section 4 - Equilibrating a coarse grained system

When generating the structure of a model, it is often the case that the generated configuration of atoms would not readily exist in equilibrium physiological conditions. Individual protein structures may be determined using X-ray crystallography Shi (2014) or cryo-EM Lyumkis (2019) techniques and thus the obtained structure data is that of the crystalline or frozen protein, or if generating structures using Alphafold, protein structure is simply a prediction learned from large databases of these crystal or cryo structures. 

If algorithmic procedures such as CHARMM-GUI, COBY, Martinize or some other scripted tool are used to generate the initial model this may lead to unnaturally ordered structures that bear little resemblance to their in vivo forms. For example; lipids may be packed incorrectly, amino acid fragments may occupy the incorrect rotameric state or proteins may need to reorient within a membrane. It is therefore necessary to perform a set of equilibration steps to bring the system to a more physiologically accurate equilibrium state before meaningful information may be extracted regarding its dynamics. This so-called equilibration phase of MD simulation is split into three distinct stages; energy minimisation, NVT and NPT.

In this part of the session, we will be running a brief energy equilibration and NVT equilibration procedure on our newly generated CG system to illustrate this process.

##### a) Energy Minimisation

The first equilibration stage is energy minimization (EM). 
The purpose of this stage is to ensure that no excessively large forces are present within the system. \
Such forces are usually caused by improper bond geometries and steric clashes between molecules; \
removing these artefacts is the primary goal of the EM phase.

These commands would typically be run within a linux terminal, but we can run them in a jupyter notebook \
using a bash interpreter (instead of python) using the %%bash command 

In [ ]:
%%bash

# define the local filenames

initial_structure="output_initial_state.pdb"
index_file="index.ndx"
topology_file="topol.top"
minimisation_mdp="mdps/min.mdp"

module load gromacs/2021.4-AVX2-GPU

gmx --version

gmx grompp -f ${minimisation_mdp} -c ${initial_structure} -p ${topology_file} -n ${index_file} -o em.tpr
gmx mdrun -deffnm em -v

In [ ]:
%%bash

# check that the output files have been generated
ls em.*

##### b) NVT equilibration

The second equilibration stage is NVT equilibration; the purpose of this phase is to \
impart velocities to the atoms within the static struture generated by EM, raising the \
system temperature to a physiologically relevant equilibrium value.  \
During NVT a temperature coupling regime is introduced into the system’s equations of \
motion. The box dimensions are held constant whilst the atomic velocities are slowly \
increased until the velocity distribution corresponds to an average system temperature \
equal to a user-defined value.  

Typically, any large biomolecules in the system are held in place using position \
restraints during this stage so as to ensure that no undesirable conformational changes \
occur to the molecules whilst the surrounding solvent molecules are allowed to \
correctly orient themselves around the larger structures. 

In [ ]:
%%bash

# define the local filenames

initial_structure="em.gro"
index_file="index.ndx"
topology_file="topol.top"
nvt_mdp="mdps/nvt.mdp"

module load gromacs/2021.4-AVX2-GPU

gmx --version

gmx grompp -f ${nvt_mdp} -c ${initial_structure} -p ${topology_file} -n ${index_file} -o nvt.tpr
gmx mdrun -deffnm nvt -v

In [ ]:
%%bash

# check that the output files have been generated
ls nvt.*

# if mdrun has run to completion, you should see a file called nvt.gro

After equilibration the system should look something like the following image

<img src="images/nvt.png" width="750">

c) NPT Equilibration

The final equilibration phase is NPT equilibration; the purpose of this phase is to take
the system generated during NVT equilibration and raise or lower its pressure to
physiological levels (i.e. 1 bar). During NPT, both a temperature and pressure coupling regime
are applied to the system’s equations of motion. As such, the previously equilibrated
temperature is held constant while the box dimensions and particle positions are
scaled until the system reaches a roughly constant average pressure. 
This is arguably the most important and nuanced stage in the equilibration process; it
is often the case with more complex systems that numerous NPT stages are required
with varied position restraints in order to allow all of the large biomolecules in the
system to relax into their equilibrium conformations without causing non-physical
deformations to the other structures in the system. This is especially the case when
dealing with large structures that directly interact with many components of the
system. \\

Once the NPT phase is complete, the equilibrated system is ready for unrestrained
production MD, the stage in which meaningful dynamical data may be extracted from
the system. Even so, when working with membranes, it is wise to ignore the first ∼
100 ns or more when performing any analysis of membrane properties.
This is required because the high density of lipids within the bilayer often take 
a considerable amount of time to find the correct organisation 
and orientation with respect to each other and any external structures embedded within the membrane.